In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os
load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv("groq_api")

D:\Langchain\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
api_key = os.getenv("google_flash")

gemini_model = ChatGoogleGenerativeAI(
    api_key=api_key,
    model="gemini-2.5-flash",
    temperature = 0.2,

)

# Ways to integrate tools:
  ## 1)@tool with init_chat_model or any other model with .bind_tools().
  ## 2)Use create_agent()

In [5]:
from langchain.tools import tool

@tool
def get_weather(location : str)->str:
    '''Getting the weather for a location'''
    return f"the weather for {location} is sunny"

In [27]:
#Using Groq:
from langchain.chat_models import init_chat_model

groq_model = init_chat_model("groq:qwen/qwen3-32b")

groq_model_with_tools = groq_model.bind_tools([get_weather])
response = groq_model_with_tools.invoke("What is the temperature in tirunelveli")

In [15]:
response.tool_calls

[{'name': 'get_weather',
  'args': {'location': 'Tirunelveli'},
  'id': '4p6pntfpt',
  'type': 'tool_call'}]

In [6]:
#Using Gemini:
from langchain.messages import SystemMessage
gemini_model_with_tools = gemini_model.bind_tools([get_weather])

gemini_response = gemini_model_with_tools.invoke([
    SystemMessage(content="You MUST use tools when answering weather questions."),
    "What is the weather in Tirunelveli?"
])

gemini_response

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"location": "Tirunelveli"}'}, '__gemini_function_call_thought_signatures__': {'56f1dec0-19e8-49fe-add7-7381e724da8c': 'CvcBAXLI2nxDpmoJ/JIOmM2m08O8ZhW3amNfNZPt0um3Z9y2ZfnC7wPEZnCYHFMgNpoUypSmVBZHrfvbQ9udUp40fnLi/oWw3qDE6pzxIn7sZkmDUHZWl4qMyd4aEtGtXeifHTxmT2xE+4txd6B0Dhmxrnq4aEQwLR3pXRnULTKP0ctjhuSnQ1KuqYt1a0v9dPlxpR+uZXH83vZ4VTWMqDKYTMHUGXFVYAW8FEVnCF5VMC0aZsMcgFhUzryCmiIU4kD4///e6bssP9fx2riAyOuS9lXa8WGkTVNM57muXuqqcXvEwNtJEO9I1oNAAiTFhaKdoZb4VXnNjw=='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bf551-8441-70d1-b7e0-60977d4b0ac2-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Tirunelveli'}, 'id': '56f1dec0-19e8-49fe-add7-7381e724da8c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 58, 'output_tokens': 73, 'total_tokens': 131, 'input_token_details': {'cache_read

In [23]:
#Printing the Response:
for tool_call in response.tool_calls:
    print(f'Tools : {tool_call['name']}')
    print(f'Args : {tool_call["args"]}')

Tools : get_weather
Args : {'location': 'Tirunelveli'}


In [29]:
messages = [{"role":"user","content":"What's the weather in Tirunelveli"}]
ai_msg = groq_model_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

final_result = groq_model_with_tools.invoke(messages)
print(final_result.text)

The weather in Tirunelveli is currently sunny.


In [9]:
messages = [{"role":"user","content":"What's the weather in Tirunelveli"}]
ai_msg = gemini_model_with_tools.invoke(messages)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

final_result = gemini_model_with_tools.invoke(messages)
print(final_result.text)

ConnectError: [Errno 11001] getaddrinfo failed